# How to Use the CityIQ API

This notebook loads and trasforms CityIQ metadata and asset events data and maps a summary of the pedestrian events over the last hour

### Import Python modules

In [1]:
# import cartoframes modules
from cartoframes.auth import set_default_credentials, Credentials
from cartoframes.viz import Map, Layer
from cartoframes.data import Dataset
from cartoframes.viz.helpers import size_continuous_layer

# import pandas and geopandas modules
import pandas as pd
import geopandas as gpd

# import CityIq and modules for timing
from cityiq import CityIq
import time
import json

### Set up CARTO credentials 

In [2]:
#load .env with credentials
from dotenv import load_dotenv
load_dotenv()

import os
BASE_URL = os.getenv("BASE_URL")
API_KEY = os.getenv("API_KEY")
USERNAME = os.getenv("USERNAME")

set_default_credentials(
    base_url=BASE_URL,
    api_key=API_KEY
)

credentials = Credentials(USERNAME, API_KEY)

### Set start and end times to pull data for the last hour

In [3]:
# set time frame for use when querying for events (epoch time in milliseconds)
endTime = int(time.time())*1000 # time when demo.py is run
startTime = endTime-3600000 # startTime is 1 hour before endTime

### Set CityIQ token and pull sensor metadata for pedestrian events types

In [4]:
# get CityIq token
myCIQ = CityIq("City")
myCIQ.fetchToken()

# getting assets - assets with PEDEVT events
myCIQ.fetchMetadata("assets","pedestrian","eventTypes:PEDEVT")
san_diego_pedestrian_sensor_metadata = myCIQ.getAssets()

*   Make a Dataframe from returned sensor metadata
*   Drop null records 
*   Split string for coordinates into latitude and longitude and store in a property
*   Make a GeoDataFrame with latitude and longitude data

In [5]:
san_diego_pedestrian_sensor_metadata_df = pd.DataFrame(san_diego_pedestrian_sensor_metadata)
san_diego_pedestrian_sensor_metadata_df.dropna(inplace = True)
# split coordinates into lat and lng 
latlng = san_diego_pedestrian_sensor_metadata_df["coordinates"].str.split(":", n = 1, expand = True) 

san_diego_pedestrian_sensor_metadata_df["latitude"]= latlng[0].astype(float)
san_diego_pedestrian_sensor_metadata_df["longitude"]= latlng[1].astype(float)

san_diego_pedestrian_sensor_metadata_gdf = gpd.GeoDataFrame(san_diego_pedestrian_sensor_metadata_df, geometry=gpd.points_from_xy(san_diego_pedestrian_sensor_metadata_df.longitude, san_diego_pedestrian_sensor_metadata_df.latitude))
san_diego_pedestrian_sensor_metadata_df.head()

,assetType,assetUid,coordinates,eventTypes,mediaType,parentAssetUid,latitude,longitude,geometry
0,CAMERA,1b576c8d-2b0c-4e44-991d-98dadc839bcc,32.7095162804853:-117.156174617791,"[PKIN, PKOUT, BICYCLE, PEDEVT]","IMAGE,VIDEO,VIDEO_STREAM",3c5f154e-97fe-4ca3-83e5-d6f542ceb434,32.709516,-117.156175,POINT (-117.156174617791 32.7095162804853)
1,CAMERA,33ac5ea9-38e0-41f1-b7d1-a8af745a0f4a,32.71091588:-117.1584351,"[PEDEVT, BICYCLE]","IMAGE,VIDEO,VIDEO_STREAM",70e8e09e-3beb-498d-9cc7-87bc8c09aea5,32.710916,-117.158435,POINT (-117.1584351 32.71091588)
2,CAMERA,56f7e069-3b22-4195-8b82-865af18ce875,32.70928915:-117.1588511,"[PKOUT, PKIN, BICYCLE, TFEVT, PEDEVT]","IMAGE,VIDEO,VIDEO_STREAM",507ffb4d-4cdd-4695-a6c1-25d53f277af2,32.709289,-117.158851,POINT (-117.1588511 32.70928915)
3,CAMERA,638d378d-d288-4d59-adf0-469ad24bde57,32.709348:-117.157252,"[PKIN, PKOUT, TFEVT, BICYCLE, PEDEVT]","IMAGE,VIDEO,VIDEO_STREAM",fad3c524-9263-4d8d-9883-83f9d4d77429,32.709348,-117.157252,POINT (-117.157252 32.709348)
4,CAMERA,6b00875c-68f6-4d2e-b48c-f98833d619a5,32.7099465891324:-117.15841682209,[PEDEVT],"IMAGE,VIDEO",d05c8ca6-7f69-4cd7-84b1-a32de3e09722,32.709947,-117.158417,POINT (-117.15841682209 32.7099465891324)


*   Loop through each sensor and request events at that sensor for the last hour.  
*   Pull nested attributes out of "properties" and "measures" properties and store in their own attribute
*   Using tabulate, create a tabular representation of the events list 

In [6]:
# empty list to collect events
san_diego_pedestrian_sensor_events_list = []


# loop through assets to fetch events for each asset
for index, row in san_diego_pedestrian_sensor_metadata_gdf.iterrows():
    myCIQ.fetchEvents("assets", row.assetUid, "PEDEVT", startTime, endTime, pageSize=500)
    assetEvents = myCIQ.getEvents()
    # pull nested attributes out of "properties" and "measures" properties and store in their own attribute
    for a in assetEvents:
        a["latitude"] = row.latitude
        a["longitude"] = row.longitude
        a["directionUnit"] = a["properties"]["directionUnit"]
        a["speedUnit"] = a["properties"]["speedUnit"]
        a["eventUid"] = a["properties"]["eventUid"]
        a["counter_direction_speed"] = a["measures"]["counter_direction_speed"]
        a["counter_direction_pedestrianCount"] = a["measures"]["counter_direction_pedestrianCount"]
        a["pedestrianCount"] = a["measures"]["pedestrianCount"]
        a["counter_direction"] = a["measures"]["counter_direction"]
        a["speed"] = a["measures"]["speed"]
        a["direction"] = a["measures"]["direction"]
        san_diego_pedestrian_sensor_events_list.append(a)

pd.DataFrame(san_diego_pedestrian_sensor_events_list).head()

,assetUid,counter_direction,counter_direction_pedestrianCount,counter_direction_speed,direction,directionUnit,eventType,eventUid,latitude,locationUid,longitude,measures,pedestrianCount,properties,speed,speedUnit,timestamp
0,1b576c8d-2b0c-4e44-991d-98dadc839bcc,94.0,0.0,0.000000,274.0,DEGREE,PEDEVT,MTU3MDc3MjIyMjE4OA==,32.709516,kvmzs7jn8srk0sm5ta4,-117.156175,"{'counter_direction_speed': 0.0, 'counter_dire...",0.0,"{'directionUnit': 'DEGREE', 'speedUnit': 'METE...",0.000000,METERS_PER_SEC,1570772222188
1,1b576c8d-2b0c-4e44-991d-98dadc839bcc,94.0,0.0,0.000000,274.0,DEGREE,PEDEVT,MTU3MDc3MjI1MjM3OQ==,32.709516,kvmzs7jn8srk0sm5ta4,-117.156175,"{'counter_direction_speed': 0.0, 'counter_dire...",0.0,"{'directionUnit': 'DEGREE', 'speedUnit': 'METE...",0.000000,METERS_PER_SEC,1570772252379
2,1b576c8d-2b0c-4e44-991d-98dadc839bcc,94.0,0.0,0.000000,274.0,DEGREE,PEDEVT,MTU3MDc3MjI4MjUwMw==,32.709516,kvmzs7jn8srk0sm5ta4,-117.156175,"{'counter_direction_speed': 0.0, 'counter_dire...",1.0,"{'directionUnit': 'DEGREE', 'speedUnit': 'METE...",2.554282,METERS_PER_SEC,1570772282503
3,1b576c8d-2b0c-4e44-991d-98dadc839bcc,94.0,3.0,3.020831,274.0,DEGREE,PEDEVT,MTU3MDc3MjMxMjY5NA==,32.709516,kvmzs7jn8srk0sm5ta4,-117.156175,"{'counter_direction_speed': 3.020831470505188,...",0.0,"{'directionUnit': 'DEGREE', 'speedUnit': 'METE...",0.000000,METERS_PER_SEC,1570772312694
4,1b576c8d-2b0c-4e44-991d-98dadc839bcc,94.0,0.0,0.000000,274.0,DEGREE,PEDEVT,MTU3MDc3MjM0MjgxOQ==,32.709516,kvmzs7jn8srk0sm5ta4,-117.156175,"{'counter_direction_speed': 0.0, 'counter_dire...",0.0,"{'directionUnit': 'DEGREE', 'speedUnit': 'METE...",0.000000,METERS_PER_SEC,1570772342819


### Make a map of graduated points sized by `pedestrianCount`

In [7]:
san_diego_pedestrian_sensor_events_df = pd.DataFrame(san_diego_pedestrian_sensor_events_list)
san_diego_pedestrian_sensor_events_df.dropna(inplace = True)

# group by location ID to get a sum of pedestrianCounts 
grouped_SD_ped_sensor_events_df = san_diego_pedestrian_sensor_events_df.groupby('assetUid').agg({'pedestrianCount': ['sum']})
grouped_SD_ped_sensor_events_df.columns = ['pedestrianCount_sum']
grouped_SD_ped_sensor_events_df.reset_index()

# join counts back to sensor metadata
merged_metadata_ped_counts_df = pd.merge(san_diego_pedestrian_sensor_metadata_df, grouped_SD_ped_sensor_events_df, on=['assetUid'])

# create a GeoDataFrame
merged_metadata_ped_counts_gdf = gpd.GeoDataFrame(merged_metadata_ped_counts_df, geometry=gpd.points_from_xy(merged_metadata_ped_counts_df.longitude, merged_metadata_ped_counts_df.latitude))

# create a CARTO dataset for upload and mapping
merged_metadata_ped_counts_d = Dataset(merged_metadata_ped_counts_gdf)

# using a helper function, make a quick map of pedestrianCount_sum
merged_metadata_ped_counts_map = Map(
    size_continuous_layer(merged_metadata_ped_counts_d, 'pedestrianCount_sum', 'Pedestrian Count')
)

# show the map
merged_metadata_ped_counts_map

### Write pedestrian count data to CARTO 

In [8]:
merged_metadata_ped_counts_d.upload(table_name='merged_metadata_ped_counts_san_diego', if_exists='replace', credentials=credentials)

/anaconda3/lib/python3.7/site-packages/carto/sql.py:229: UserWarning: Batch SQL job created with job_id: 82087cfe-3ad3-4eec-9046-94e999bdfeeb
  warnings.warn('Batch SQL job created with job_id: {job_id}'.format(job_id=data['job_id']))


### Publish pedestrian count data map

In [10]:
merged_metadata_ped_counts_map.sync_data('merged_metadata_ped_counts_map')
merged_metadata_ped_counts_map.publish('merged_metadata_ped_counts_map', maps_api_key=API_KEY)

/anaconda3/lib/python3.7/site-packages/carto/sql.py:229: UserWarning: Batch SQL job created with job_id: f9510b3e-98a4-478a-87f9-504dfd0a691f
  warnings.warn('Batch SQL job created with job_id: {job_id}'.format(job_id=data['job_id']))
/anaconda3/lib/python3.7/site-packages/cartoframes/viz/kuviz.py:65: UserWarning: Table `merged_metadata_ped_counts_map_1` created. In order to publish the map, you will need to create a new Regular API key with permissions to Maps API and the table `merged_metadata_ped_counts_map_1`. You can do it from your CARTO dashboard or using the Auth API. You can get more info at https://carto.com/developers/auth-api/guides/types-of-API-Keys/
  'https://carto.com/developers/auth-api/guides/types-of-API-Keys/'.format(table_name, table_name))


{'id': '5c52f132-08c7-4a32-bc88-1d9ff31132d6',
 'url': 'https://usignite.carto.com/u/usignite-admin/kuviz/5c52f132-08c7-4a32-bc88-1d9ff31132d6',
 'name': 'merged_metadata_ped_counts_map',
 'privacy': 'public'}

### pull sensor metadata for parking events types

In [9]:
myCIQ.fetchMetadata("assets","parking","eventTypes:PKIN")
san_diego_parking_sensor_metadata = myCIQ.getAssets()

san_diego_parking_sensor_metadata_df = pd.DataFrame(san_diego_parking_sensor_metadata)
san_diego_parking_sensor_metadata_df.dropna(inplace = True)
# split coordinates into lat and lng 
latlng = san_diego_parking_sensor_metadata_df["coordinates"].str.split(":", n = 1, expand = True) 

san_diego_parking_sensor_metadata_df["latitude"]= latlng[0].astype(float)
san_diego_parking_sensor_metadata_df["longitude"]= latlng[1].astype(float)

san_diego_parking_sensor_metadata_gdf = gpd.GeoDataFrame(san_diego_parking_sensor_metadata_df, geometry=gpd.points_from_xy(san_diego_parking_sensor_metadata_df.longitude, san_diego_parking_sensor_metadata_df.latitude))
san_diego_parking_sensor_metadata_df.head()

More Metadata assets are available for this query.
TotalElements: 520. Your specified size: 100


,assetType,assetUid,coordinates,eventTypes,mediaType,parentAssetUid,latitude,longitude,geometry
0,CAMERA,000223ee-a868-474b-abcb-12ff1bad00a3,32.71143062:-117.1600173,"[PKOUT, PKIN, TFEVT, BICYCLE]","IMAGE,VIDEO",131d67b7-daaf-4252-9202-5df347a6c2a3,32.711431,-117.160017,POINT (-117.1600173 32.71143062)
1,CAMERA,00c6b5d6-df99-4696-be16-d274af05ea89,32.71268097:-117.1543763,"[PKOUT, TFEVT, PKIN, BICYCLE]","IMAGE,VIDEO,VIDEO_STREAM",21ec1d88-a4ec-4b2d-882e-a23477e30bb6,32.712681,-117.154376,POINT (-117.1543763 32.71268097)
2,CAMERA,0161240c-aa9c-4a60-b693-d7af5270ebe3,32.71351413:-117.1561822,"[PKOUT, PKIN, BICYCLE, TFEVT]","IMAGE,VIDEO",afa710e2-a43e-427c-a27d-7887cd000845,32.713514,-117.156182,POINT (-117.1561822 32.71351413)
3,CAMERA,02276cc4-dc67-4e12-ab0c-ffe3d19747c4,32.7154760627819:-117.152087034359,"[PKIN, PKOUT, TFEVT, BICYCLE]","IMAGE,VIDEO",7ae6bb48-d9a9-4856-b0f2-de1a22233c53,32.715476,-117.152087,POINT (-117.152087034359 32.7154760627819)
4,CAMERA,02f6bdcb-23ec-4fb0-8c0f-5a1c43ada674,32.70948287:-117.1576975,"[PKIN, PKOUT]","IMAGE,VIDEO",a8f5032b-ab15-465b-86e1-5ae4be74f541,32.709483,-117.157697,POINT (-117.1576975 32.70948287)


### Fetch parking events

In [10]:
# empty list to collect events
san_diego_parking_sensor_events_list = []


# loop through assets to fetch events for each asset
for index, row in san_diego_parking_sensor_metadata_gdf.iterrows():
    myCIQ.fetchEvents("assets", row.assetUid, "PKIN", startTime, endTime, pageSize=500)
    assetEvents = myCIQ.getEvents()
    # pull nested attributes out of "properties" and "measures" properties and store in their own attribute
    for a in assetEvents:
        a["latitude"] = row.latitude
        a["longitude"] = row.longitude
        san_diego_parking_sensor_events_list.append(a)

pd.DataFrame(san_diego_parking_sensor_events_list).head()

,assetUid,eventType,latitude,locationUid,longitude,measures,properties,timestamp
0,000223ee-a868-474b-abcb-12ff1bad00a3,PKIN,32.711431,x10g3qw5mojj767m7t,-117.160017,{},"{'orgPixelCoordinates': '1169.9373:341.96188,1...",1570770506033
1,000223ee-a868-474b-abcb-12ff1bad00a3,PKIN,32.711431,7swwraqa8o8jj76a8a3,-117.160017,{},"{'orgPixelCoordinates': '1389.0879:673.66766,1...",1570771760523
2,000223ee-a868-474b-abcb-12ff1bad00a3,PKIN,32.711431,x10g3qw5mojj767m7t,-117.160017,{},"{'orgPixelCoordinates': '1186.5763:335.6384,13...",1570771800977
3,000223ee-a868-474b-abcb-12ff1bad00a3,PKIN,32.711431,7swwraqa8o8jj76a8a3,-117.160017,{},"{'orgPixelCoordinates': '1424.8761:662.11816,1...",1570772549218
4,000223ee-a868-474b-abcb-12ff1bad00a3,PKIN,32.711431,x10g3qw5mojj767m7t,-117.160017,{},"{'orgPixelCoordinates': '1267.0674:327.02667,1...",1570773753789


### Pull sensor metadata for traffic

In [11]:
myCIQ.fetchMetadata("assets","traffic","eventTypes:TFEVT")
san_diego_traffic_sensor_metadata = myCIQ.getAssets()

san_diego_traffic_sensor_metadata_df = pd.DataFrame(san_diego_traffic_sensor_metadata)
san_diego_traffic_sensor_metadata_df.dropna(inplace = True)
# split coordinates into lat and lng 
latlng = san_diego_traffic_sensor_metadata_df["coordinates"].str.split(":", n = 1, expand = True) 

san_diego_traffic_sensor_metadata_df["latitude"]= latlng[0].astype(float)
san_diego_traffic_sensor_metadata_df["longitude"]= latlng[1].astype(float)

san_diego_traffic_sensor_metadata_gdf = gpd.GeoDataFrame(san_diego_traffic_sensor_metadata_df, geometry=gpd.points_from_xy(san_diego_traffic_sensor_metadata_df.longitude, san_diego_traffic_sensor_metadata_df.latitude))
san_diego_traffic_sensor_metadata_df.head()

More Metadata assets are available for this query.
TotalElements: 334. Your specified size: 100


,assetType,assetUid,coordinates,eventTypes,mediaType,parentAssetUid,latitude,longitude,geometry
0,CAMERA,000223ee-a868-474b-abcb-12ff1bad00a3,32.71143062:-117.1600173,"[PKOUT, PKIN, TFEVT, BICYCLE]","IMAGE,VIDEO",131d67b7-daaf-4252-9202-5df347a6c2a3,32.711431,-117.160017,POINT (-117.1600173 32.71143062)
1,CAMERA,00c6b5d6-df99-4696-be16-d274af05ea89,32.71268097:-117.1543763,"[PKOUT, TFEVT, PKIN, BICYCLE]","IMAGE,VIDEO,VIDEO_STREAM",21ec1d88-a4ec-4b2d-882e-a23477e30bb6,32.712681,-117.154376,POINT (-117.1543763 32.71268097)
2,CAMERA,0161240c-aa9c-4a60-b693-d7af5270ebe3,32.71351413:-117.1561822,"[PKOUT, PKIN, BICYCLE, TFEVT]","IMAGE,VIDEO",afa710e2-a43e-427c-a27d-7887cd000845,32.713514,-117.156182,POINT (-117.1561822 32.71351413)
3,CAMERA,02276cc4-dc67-4e12-ab0c-ffe3d19747c4,32.7154760627819:-117.152087034359,"[PKIN, PKOUT, TFEVT, BICYCLE]","IMAGE,VIDEO",7ae6bb48-d9a9-4856-b0f2-de1a22233c53,32.715476,-117.152087,POINT (-117.152087034359 32.7154760627819)
4,CAMERA,02fd833c-5d93-45f3-99fc-844c492f3067,32.7128702:-117.1573235,"[TFEVT, PKOUT, BICYCLE, PKIN]","IMAGE,VIDEO",40b6bd95-386f-4970-8bc5-ee852c509fc1,32.712870,-117.157324,POINT (-117.1573235 32.7128702)


### Fetch traffic events

In [12]:
# empty list to collect events
san_diego_traffic_sensor_events_list = []


# loop through assets to fetch events for each asset
for index, row in san_diego_traffic_sensor_metadata_gdf.iterrows():
    myCIQ.fetchEvents("assets", row.assetUid, "PKIN", startTime, endTime, pageSize=500)
    assetEvents = myCIQ.getEvents()
    # pull nested attributes out of "properties" and "measures" properties and store in their own attribute
    for a in assetEvents:
        a["latitude"] = row.latitude
        a["longitude"] = row.longitude
        san_diego_traffic_sensor_events_list.append(a)

pd.DataFrame(san_diego_traffic_sensor_events_list).head()

,assetUid,eventType,latitude,locationUid,longitude,measures,properties,timestamp
0,000223ee-a868-474b-abcb-12ff1bad00a3,PKIN,32.711431,x10g3qw5mojj767m7t,-117.160017,{},"{'orgPixelCoordinates': '1169.9373:341.96188,1...",1570770506033
1,000223ee-a868-474b-abcb-12ff1bad00a3,PKIN,32.711431,7swwraqa8o8jj76a8a3,-117.160017,{},"{'orgPixelCoordinates': '1389.0879:673.66766,1...",1570771760523
2,000223ee-a868-474b-abcb-12ff1bad00a3,PKIN,32.711431,x10g3qw5mojj767m7t,-117.160017,{},"{'orgPixelCoordinates': '1186.5763:335.6384,13...",1570771800977
3,000223ee-a868-474b-abcb-12ff1bad00a3,PKIN,32.711431,7swwraqa8o8jj76a8a3,-117.160017,{},"{'orgPixelCoordinates': '1424.8761:662.11816,1...",1570772549218
4,000223ee-a868-474b-abcb-12ff1bad00a3,PKIN,32.711431,x10g3qw5mojj767m7t,-117.160017,{},"{'orgPixelCoordinates': '1267.0674:327.02667,1...",1570773753789


### Pull sensor metadata for bikes


In [14]:
myCIQ.fetchMetadata("assets","bicycle","eventTypes:BICYCLE")
san_diego_bike_sensor_metadata = myCIQ.getAssets()

san_diego_bike_sensor_metadata_df = pd.DataFrame(san_diego_bike_sensor_metadata)
san_diego_bike_sensor_metadata_df.head()
san_diego_bike_sensor_metadata_df.dropna(inplace = True)
# split coordinates into lat and lng 
latlng = san_diego_bike_sensor_metadata_df["coordinates"].str.split(":", n = 1, expand = True) 
san_diego_bike_sensor_metadata_df["latitude"]= latlng[0].astype(float)
san_diego_bike_sensor_metadata_df["longitude"]= latlng[1].astype(float)

san_diego_bike_sensor_metadata_gdf = gpd.GeoDataFrame(san_diego_bike_sensor_metadata_df, geometry=gpd.points_from_xy(san_diego_bike_sensor_metadata_df.longitude, san_diego_bike_sensor_metadata_df.latitude))
san_diego_bike_sensor_metadata_df.head()

Error when fetching metadata
https://sandiego.cityiq.io/api/v2/metadata//assets/search
<Response [403]> 




,assetType,assetUid,coordinates,eventTypes,mediaType,parentAssetUid,latitude,longitude,geometry
0,CAMERA,000223ee-a868-474b-abcb-12ff1bad00a3,32.71143062:-117.1600173,"[PKOUT, PKIN, TFEVT, BICYCLE]","IMAGE,VIDEO",131d67b7-daaf-4252-9202-5df347a6c2a3,32.711431,-117.160017,POINT (-117.1600173 32.71143062)
1,CAMERA,00c6b5d6-df99-4696-be16-d274af05ea89,32.71268097:-117.1543763,"[PKOUT, TFEVT, PKIN, BICYCLE]","IMAGE,VIDEO,VIDEO_STREAM",21ec1d88-a4ec-4b2d-882e-a23477e30bb6,32.712681,-117.154376,POINT (-117.1543763 32.71268097)
2,CAMERA,0161240c-aa9c-4a60-b693-d7af5270ebe3,32.71351413:-117.1561822,"[PKOUT, PKIN, BICYCLE, TFEVT]","IMAGE,VIDEO",afa710e2-a43e-427c-a27d-7887cd000845,32.713514,-117.156182,POINT (-117.1561822 32.71351413)
3,CAMERA,02276cc4-dc67-4e12-ab0c-ffe3d19747c4,32.7154760627819:-117.152087034359,"[PKIN, PKOUT, TFEVT, BICYCLE]","IMAGE,VIDEO",7ae6bb48-d9a9-4856-b0f2-de1a22233c53,32.715476,-117.152087,POINT (-117.152087034359 32.7154760627819)
4,CAMERA,02fd833c-5d93-45f3-99fc-844c492f3067,32.7128702:-117.1573235,"[TFEVT, PKOUT, BICYCLE, PKIN]","IMAGE,VIDEO",40b6bd95-386f-4970-8bc5-ee852c509fc1,32.712870,-117.157324,POINT (-117.1573235 32.7128702)


### Fetch bike data

In [15]:
# empty list to collect events
san_diego_bike_sensor_events_list = []


# loop through assets to fetch events for each asset
for index, row in san_diego_bike_sensor_metadata_gdf.iterrows():
    myCIQ.fetchEvents("assets", row.assetUid, "BICYCLE", startTime, endTime, pageSize=500)
    assetEvents = myCIQ.getEvents()
    # pull nested attributes out of "properties" and "measures" properties and store in their own attribute
    for a in assetEvents:
        a["latitude"] = row.latitude
        a["longitude"] = row.longitude
        san_diego_bike_sensor_events_list.append(a)


pd.DataFrame(san_diego_bike_sensor_events_list).head()

fetchEvents Failed
<Response [403]>
'content'
fetchEvents Failed
<Response [403]>
'content'
fetchEvents Failed
<Response [403]>
'content'
fetchEvents Failed
<Response [403]>
'content'
fetchEvents Failed
<Response [403]>
'content'
fetchEvents Failed
<Response [403]>
'content'
fetchEvents Failed
<Response [403]>
'content'
fetchEvents Failed
<Response [403]>
'content'
fetchEvents Failed
<Response [403]>
'content'
fetchEvents Failed
<Response [403]>
'content'
fetchEvents Failed
<Response [403]>
'content'
fetchEvents Failed
<Response [403]>
'content'
fetchEvents Failed
<Response [403]>
'content'
fetchEvents Failed
<Response [403]>
'content'
fetchEvents Failed
<Response [403]>
'content'
fetchEvents Failed
<Response [403]>
'content'
fetchEvents Failed
<Response [403]>
'content'
fetchEvents Failed
<Response [403]>
'content'
fetchEvents Failed
<Response [403]>
'content'
fetchEvents Failed
<Response [403]>
'content'
fetchEvents Failed
<Response [403]>
'content'
fetchEvents Failed
<Response [403]

,assetUid,eventType,latitude,locationUid,longitude,measures,properties,timestamp
0,47e551ed-625d-4013-ac1d-32c4b8233240,PKIN,32.712945,b9owvajg1fdjkmlobh8,-117.152753,{},"{'orgPixelCoordinates': '703.53265:271.2701,77...",1570772293553
1,47e551ed-625d-4013-ac1d-32c4b8233240,PKIN,32.712945,b9owvajg1fdjkmlobh8,-117.152753,{},"{'orgPixelCoordinates': '1370.4467:313.39752,1...",1570772991007
2,47e551ed-625d-4013-ac1d-32c4b8233240,PKIN,32.712945,b9owvajg1fdjkmlobh8,-117.152753,{},"{'orgPixelCoordinates': '1244.6517:309.78238,1...",1570773061785
3,47e551ed-625d-4013-ac1d-32c4b8233240,PKIN,32.712945,b9owvajg1fdjkmlobh8,-117.152753,{},"{'orgPixelCoordinates': '703.53265:271.2701,77...",1570772293553
4,47e551ed-625d-4013-ac1d-32c4b8233240,PKIN,32.712945,b9owvajg1fdjkmlobh8,-117.152753,{},"{'orgPixelCoordinates': '1370.4467:313.39752,1...",1570772991007
